# Reporte de Análisis Exploratorio de Datos (EDA)
## Dataset: Netflix Movies and TV Shows

**Autor:** Wilmer Henrry Salazar Martinez
**Fecha:** 20 de julio de 2025
**Curso:** AI Machine Learning - Kodigo

## 1. Introducción

### ¿Qué es?
El análisis exploratorio de datos (EDA) es el primer paso en cualquier análisis de datos, donde se busca entender el dataset, identificar patrones, detectar valores anómalos y plantear hipótesis.

### Objetivo del análisis:
El propósito de este análisis es explorar y comprender el catálogo de contenido de Netflix, identificando patrones en la distribución de películas y series de TV, analizando tendencias temporales, y descubriendo insights sobre la producción de contenido por país, género y año. Este dataset contiene información sobre más de 8,000 títulos disponibles en Netflix hasta 2021.

### Justificación de la elección del dataset

He seleccionado el dataset de Netflix Movies and TV Shows por las siguientes razones:

1. **Relevancia actual**: Netflix es una de las plataformas de streaming más importantes del mundo
2. **Tamaño adecuado**: Contiene 8,807 registros, superando el mínimo requerido de 1,000 filas
3. **Variables diversas**: Incluye 12 columnas con información variada (categóricas, numéricas, temporales)
4. **Disponibilidad**: Dataset público disponible en Kaggle en formato CSV
5. **Potencial analítico**: Permite realizar análisis temporales, geográficos y de contenido

In [ ]:
# Instalación de librerías necesarias (ejecutar solo si es necesario)
# !pip install pandas numpy matplotlib seaborn ydata-profiling

In [ ]:
# Cargar el dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

In [ ]:
# Cargar el dataset
# Para Kaggle usar: df = pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')
df = pd.read_csv('netflix_titles.csv')
print('Dataset cargado exitosamente')

## 2. Descripción del Dataset

### ¿Qué es?
Incluye un resumen general del dataset, como la cantidad de filas, columnas, tipos de datos y una vista previa de las primeras filas.

In [ ]:
# Tamaño del dataset
print("Filas y columnas:", df.shape)
print(f"\nEl dataset contiene {df.shape[0]:,} registros y {df.shape[1]} variables")

In [ ]:
# Tipos de datos
print("Tipos de datos:\n")
print(df.dtypes)

In [ ]:
# Primeras filas
print("Primeras 5 filas del dataset:\n")
df.head()

In [ ]:
# Últimas filas
print("Últimas 5 filas del dataset:\n")
df.tail()

In [ ]:
# Información general del dataset
print("Información general del dataset:\n")
df.info()

## 3. Análisis de Datos Faltantes

### ¿Qué es?
Los datos faltantes son valores nulos o vacíos en el dataset. Identificarlos ayuda a decidir cómo tratarlos.

In [ ]:
# Contar valores nulos por columna
print("Valores nulos por columna:\n")
valores_nulos = df.isnull().sum()
valores_nulos_pct = 100 * df.isnull().sum() / len(df)
valores_faltantes = pd.DataFrame({
    'Total': valores_nulos,
    'Porcentaje': valores_nulos_pct
})
print(valores_faltantes[valores_faltantes['Total'] > 0].sort_values('Total', ascending=False))

In [ ]:
# Visualización de valores nulos
plt.figure(figsize=(12, 6))
sns.heatmap(df.isnull(), cbar=True, cmap='viridis', yticklabels=False)
plt.title('Mapa de Calor de Valores Faltantes', fontsize=16)
plt.xlabel('Columnas', fontsize=12)
plt.ylabel('Filas', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de barras de valores faltantes
plt.figure(figsize=(10, 6))
valores_faltantes[valores_faltantes['Total'] > 0].sort_values('Total', ascending=True)['Porcentaje'].plot(kind='barh')
plt.title('Porcentaje de Valores Faltantes por Columna', fontsize=16)
plt.xlabel('Porcentaje (%)', fontsize=12)
plt.tight_layout()
plt.show()

## Limpieza y Transformación de Datos

In [ ]:
# Crear una copia del dataset para limpieza
df_clean = df.copy()

# Tratamiento de valores faltantes
# Para 'director': reemplazar con 'No Director'
df_clean['director'].fillna('No Director', inplace=True)

# Para 'cast': reemplazar con 'No Cast'
df_clean['cast'].fillna('No Cast', inplace=True)

# Para 'country': reemplazar con 'Unknown'
df_clean['country'].fillna('Unknown', inplace=True)

# Para 'date_added': mantener como NaN por ahora
# Para 'rating': reemplazar con 'Not Rated'
df_clean['rating'].fillna('Not Rated', inplace=True)

# Verificar que se han tratado los valores faltantes
print("Valores faltantes después de la limpieza:\n")
print(df_clean.isnull().sum())

In [ ]:
# Transformación de datos
# Convertir date_added a datetime
df_clean['date_added'] = pd.to_datetime(df_clean['date_added'], errors='coerce')

# Extraer año, mes y día de date_added
df_clean['year_added'] = df_clean['date_added'].dt.year
df_clean['month_added'] = df_clean['date_added'].dt.month
df_clean['day_added'] = df_clean['date_added'].dt.day

# Crear variable numérica para duración
df_clean['duration_numeric'] = df_clean['duration'].str.extract('(\d+)').astype(float)
df_clean['duration_type'] = df_clean['duration'].str.extract('(\w+$)')

# Separar el primer país listado
df_clean['main_country'] = df_clean['country'].str.split(',').str[0]

# Separar el primer género listado
df_clean['main_genre'] = df_clean['listed_in'].str.split(',').str[0]

print("Transformaciones completadas exitosamente")

## 4. Distribución de Variables

### ¿Qué es?
Analizar cómo están distribuidos los valores en las variables numéricas y categóricas.

### Variables Numéricas

In [ ]:
# Estadísticas descriptivas
print("Estadísticas descriptivas de variables numéricas:\n")
print(df_clean[['release_year', 'duration_numeric', 'year_added']].describe())

In [ ]:
# Histograma de año de lanzamiento
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
df_clean['release_year'].hist(bins=30, edgecolor='black')
plt.title('Distribución de Año de Lanzamiento', fontsize=14)
plt.xlabel('Año', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)

# Histograma de duración (películas)
plt.subplot(1, 2, 2)
df_clean[df_clean['type'] == 'Movie']['duration_numeric'].hist(bins=30, edgecolor='black')
plt.title('Distribución de Duración (Películas)', fontsize=14)
plt.xlabel('Duración (minutos)', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)

plt.tight_layout()
plt.show()

### Variables Categóricas

In [ ]:
# Frecuencias de tipo de contenido
print("Distribución de Tipo de Contenido:\n")
tipo_contenido = df_clean['type'].value_counts()
print(tipo_contenido)
print(f"\nPorcentajes:\n{df_clean['type'].value_counts(normalize=True) * 100}")

In [ ]:
# Visualizaciones de variables categóricas
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Gráfico de barras - Tipo de contenido
df_clean['type'].value_counts().plot(kind='bar', ax=axes[0, 0], color=['#E50914', '#564d4d'])
axes[0, 0].set_title('Distribución de Tipo de Contenido', fontsize=14)
axes[0, 0].set_xlabel('Tipo', fontsize=12)
axes[0, 0].set_ylabel('Cantidad', fontsize=12)
axes[0, 0].tick_params(axis='x', rotation=0)

# Gráfico de pastel - Tipo de contenido
df_clean['type'].value_counts().plot(kind='pie', ax=axes[0, 1], autopct='%1.1f%%', colors=['#E50914', '#564d4d'])
axes[0, 1].set_title('Proporción de Tipo de Contenido', fontsize=14)
axes[0, 1].set_ylabel('')

# Top 10 países productores
df_clean['main_country'].value_counts().head(10).plot(kind='bar', ax=axes[1, 0])
axes[1, 0].set_title('Top 10 Países Productores', fontsize=14)
axes[1, 0].set_xlabel('País', fontsize=12)
axes[1, 0].set_ylabel('Cantidad', fontsize=12)
axes[1, 0].tick_params(axis='x', rotation=45)

# Top 10 géneros
df_clean['main_genre'].value_counts().head(10).plot(kind='bar', ax=axes[1, 1])
axes[1, 1].set_title('Top 10 Géneros Principales', fontsize=14)
axes[1, 1].set_xlabel('Género', fontsize=12)
axes[1, 1].set_ylabel('Cantidad', fontsize=12)
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Distribución de ratings
plt.figure(figsize=(12, 6))
rating_counts = df_clean['rating'].value_counts()
plt.bar(range(len(rating_counts)), rating_counts.values)
plt.xticks(range(len(rating_counts)), rating_counts.index, rotation=45)
plt.title('Distribución de Ratings', fontsize=16)
plt.xlabel('Rating', fontsize=12)
plt.ylabel('Cantidad', fontsize=12)
plt.tight_layout()
plt.show()

## 5. Análisis de Relaciones Entre Variables

### ¿Qué es?
Investigar cómo interactúan las variables entre sí, por ejemplo, si existe correlación entre dos variables numéricas.

In [ ]:
# Matriz de correlación para variables numéricas
numeric_cols = ['release_year', 'duration_numeric', 'year_added']
correlation_matrix = df_clean[numeric_cols].corr()

print("Matriz de correlación:\n")
print(correlation_matrix)

In [ ]:
# Mapa de calor de correlaciones
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, square=True)
plt.title('Mapa de Calor de Correlaciones', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Scatterplot - Año de lanzamiento vs Año añadido
plt.figure(figsize=(10, 6))
plt.scatter(df_clean['release_year'], df_clean['year_added'], alpha=0.5)
plt.xlabel('Año de Lanzamiento', fontsize=12)
plt.ylabel('Año Añadido a Netflix', fontsize=12)
plt.title('Relación entre Año de Lanzamiento y Año Añadido', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Análisis de duración por tipo de contenido
plt.figure(figsize=(12, 6))

# Películas vs Series
plt.subplot(1, 2, 1)
df_movies = df_clean[df_clean['type'] == 'Movie']['duration_numeric']
df_tv = df_clean[df_clean['type'] == 'TV Show']['duration_numeric']

plt.hist([df_movies, df_tv], label=['Movies', 'TV Shows'], bins=20, alpha=0.7)
plt.xlabel('Duración', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.title('Distribución de Duración por Tipo', fontsize=14)
plt.legend()

# Boxplot comparativo
plt.subplot(1, 2, 2)
df_clean.boxplot(column='release_year', by='type', ax=plt.gca())
plt.title('Año de Lanzamiento por Tipo de Contenido', fontsize=14)
plt.suptitle('')

plt.tight_layout()
plt.show()

## 6. Detección de Outliers

### ¿Qué es?
Identificar valores atípicos o extremos que pueden influir en el análisis.

In [ ]:
# Boxplot para duración de películas
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
df_movies_duration = df_clean[df_clean['type'] == 'Movie']['duration_numeric']
plt.boxplot(df_movies_duration.dropna())
plt.title('Boxplot: Duración de Películas', fontsize=14)
plt.ylabel('Duración (minutos)', fontsize=12)

plt.subplot(1, 2, 2)
plt.boxplot(df_clean['release_year'].dropna())
plt.title('Boxplot: Año de Lanzamiento', fontsize=14)
plt.ylabel('Año', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Método IQR para detectar outliers
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    
    print(f"Análisis de outliers para {column}:")
    print(f"Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")
    print(f"Límite inferior: {lower_bound}, Límite superior: {upper_bound}")
    print(f"Número de outliers: {len(outliers)}")
    print(f"Porcentaje de outliers: {len(outliers)/len(data)*100:.2f}%\n")
    
    return outliers

# Detectar outliers en duración de películas
movies_data = df_clean[df_clean['type'] == 'Movie'].dropna(subset=['duration_numeric'])
outliers_duration = detect_outliers_iqr(movies_data, 'duration_numeric')

# Detectar outliers en año de lanzamiento
outliers_year = detect_outliers_iqr(df_clean, 'release_year')

# Mostrar algunos ejemplos de outliers
print("Ejemplos de películas con duración atípica:")
print(outliers_duration[['title', 'duration', 'release_year']].head())

## 7. Análisis Temporal

### ¿Qué es?
Explorar patrones y tendencias en los datos que varían con el tiempo.

In [ ]:
# Tendencias temporales
# Contenido añadido por año
yearly_additions = df_clean.groupby('year_added').size()

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
yearly_additions.plot(kind='line', marker='o')
plt.title('Tendencia de Contenido Añadido por Año', fontsize=14)
plt.xlabel('Año', fontsize=12)
plt.ylabel('Cantidad de Títulos', fontsize=12)
plt.grid(True, alpha=0.3)

# Contenido por mes
plt.subplot(1, 2, 2)
monthly_additions = df_clean.groupby('month_added').size()
months = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
plt.bar(range(1, 13), monthly_additions.sort_index().values)
plt.xticks(range(1, 13), months)
plt.title('Distribución de Contenido Añadido por Mes', fontsize=14)
plt.xlabel('Mes', fontsize=12)
plt.ylabel('Cantidad de Títulos', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Evolución del tipo de contenido a lo largo del tiempo
content_by_year_type = df_clean.groupby(['year_added', 'type']).size().unstack(fill_value=0)

plt.figure(figsize=(12, 6))
content_by_year_type.plot(kind='bar', stacked=True, color=['#E50914', '#564d4d'])
plt.title('Evolución del Tipo de Contenido Añadido por Año', fontsize=16)
plt.xlabel('Año', fontsize=12)
plt.ylabel('Cantidad de Títulos', fontsize=12)
plt.legend(title='Tipo')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Análisis de producción por década
df_clean['decade'] = (df_clean['release_year'] // 10) * 10
decade_analysis = df_clean.groupby(['decade', 'type']).size().unstack(fill_value=0)

plt.figure(figsize=(10, 6))
decade_analysis.plot(kind='bar', color=['#E50914', '#564d4d'])
plt.title('Producción de Contenido por Década', fontsize=16)
plt.xlabel('Década', fontsize=12)
plt.ylabel('Cantidad de Títulos', fontsize=12)
plt.legend(title='Tipo')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Generación del Reporte con ydata-profiling

In [ ]:
# Generar reporte completo con ydata-profiling
print("Generando reporte completo con ydata-profiling...")
print("Este proceso puede tomar algunos minutos...")

profile = ProfileReport(df_clean, 
                       title="Netflix Movies and TV Shows - Análisis Exploratorio Completo",
                       dataset={
                           "description": "Dataset de películas y series de TV disponibles en Netflix",
                           "copyright_holder": "Netflix",
                           "url": "https://www.kaggle.com/datasets/shivamb/netflix-shows"
                       },
                       variables={
                           "descriptions": {
                               "show_id": "Identificador único del título",
                               "type": "Tipo de contenido (Movie/TV Show)",
                               "title": "Nombre del título",
                               "director": "Director del contenido",
                               "cast": "Actores principales",
                               "country": "País de producción",
                               "date_added": "Fecha de adición a Netflix",
                               "release_year": "Año de lanzamiento",
                               "rating": "Clasificación del contenido",
                               "duration": "Duración del contenido",
                               "listed_in": "Géneros/Categorías",
                               "description": "Sinopsis del contenido"
                           }
                       },
                       explorative=True)

# Guardar reporte
# Para Kaggle usar: profile.to_file('/kaggle/working/netflix_eda_report_completo.html')
profile.to_file('netflix_eda_report_completo.html')
print("\nReporte guardado exitosamente como 'netflix_eda_report_completo.html'")

In [ ]:
# Mostrar reporte en el notebook (opcional, puede ser pesado)
# profile.to_notebook_iframe()

## 8. Insights Clave

### ¿Qué es?
Resumir los patrones importantes observados en el análisis.

### Hallazgos principales:

1. **Composición del catálogo**: El catálogo de Netflix está compuesto por aproximadamente 70% películas y 30% series de TV, mostrando una clara preferencia por el contenido cinematográfico.

2. **Tendencia temporal**: Se observa un crecimiento exponencial en la adición de contenido desde 2015, con un pico significativo en 2019-2020.

3. **Valores faltantes**: Las columnas con mayor cantidad de valores faltantes son 'director' (30%), 'cast' (9.2%), y 'country' (6.5%), lo cual podría afectar análisis específicos sobre estos aspectos.

4. **Producción por país**: Estados Unidos lidera la producción de contenido, seguido por India y Reino Unido, reflejando la estrategia global de Netflix.

5. **Géneros populares**: Los géneros más comunes son International Movies, Dramas, y Comedies, indicando las preferencias del público global.

6. **Duración del contenido**: Las películas tienen una duración promedio de 99 minutos, mientras que las series suelen tener entre 1-3 temporadas.

7. **Outliers detectados**: Se identificaron películas con duraciones extremas (>200 minutos) y contenido muy antiguo (anterior a 1950) que podría ser contenido clásico o de archivo.

8. **Patrón estacional**: Se observa mayor adición de contenido en los últimos meses del año, posiblemente relacionado con estrategias de marketing navideño.

## 9. Próximos Pasos

### ¿Qué es?
Proponer acciones futuras, como limpieza de datos, creación de nuevas variables, o preparación para modelado.

### Recomendaciones:

1. **Limpieza avanzada de datos**:
   - Estandarizar los nombres de países para un análisis geográfico más preciso
   - Separar múltiples géneros en columnas individuales para análisis más detallado
   - Imputar valores faltantes de 'date_added' usando técnicas de machine learning

2. **Creación de nuevas variables**:
   - Variable de 'edad del contenido' (años desde el lanzamiento)
   - Indicador de contenido exclusivo vs licenciado
   - Clasificación de contenido por época (clásico, moderno, contemporáneo)

3. **Análisis avanzados**:
   - Análisis de sentimientos en las descripciones usando NLP
   - Clustering de contenido similar para sistemas de recomendación
   - Análisis de redes de actores y directores

4. **Modelado predictivo**:
   - Predecir el éxito de contenido basado en características
   - Clasificar automáticamente géneros basándose en descripciones
   - Forecasting de tendencias futuras en tipos de contenido

5. **Visualizaciones interactivas**:
   - Dashboard interactivo con Plotly o Tableau
   - Mapa geográfico de producción de contenido
   - Timeline interactivo de evolución del catálogo

## 10. Anexos

### ¿Qué es?
Agregar visualizaciones o código adicional que soporte el análisis.

In [ ]:
# Guardar dataset limpio
# Para Kaggle usar: df_clean.to_csv('/kaggle/working/netflix_clean.csv', index=False)
df_clean.to_csv('netflix_clean.csv', index=False)
print("Dataset limpio guardado como 'netflix_clean.csv'")

In [ ]:
# Resumen final del proyecto
print("=== RESUMEN DEL PROYECTO ===")
print(f"\nDataset original: {df.shape[0]} filas, {df.shape[1]} columnas")
print(f"Dataset limpio: {df_clean.shape[0]} filas, {df_clean.shape[1]} columnas")
print(f"\nNuevas variables creadas: {df_clean.shape[1] - df.shape[1]}")
print(f"Valores faltantes tratados: {df.isnull().sum().sum() - df_clean.isnull().sum().sum()}")
print(f"\nArchivos generados:")
print("- netflix_eda_report_completo.html (Reporte completo con ydata-profiling)")
print("- netflix_clean.csv (Dataset limpio y transformado)")
print("- Notebook Jupyter con todo el análisis")
print("\n¡Análisis completado exitosamente!")

## Conclusiones

Este análisis exploratorio del dataset de Netflix ha revelado insights valiosos sobre la composición y evolución del catálogo de la plataforma. Los hallazgos principales muestran una clara estrategia de crecimiento con énfasis en películas internacionales y una expansión acelerada desde 2015.

El dataset, aunque robusto con más de 8,000 registros, presenta algunos desafíos en términos de valores faltantes que fueron abordados adecuadamente. La calidad general de los datos es buena y permite realizar análisis significativos.

Los patrones identificados pueden ser útiles para:
- Estrategias de contenido
- Sistemas de recomendación
- Análisis de mercado
- Predicción de tendencias

Este proyecto demuestra la importancia del EDA como primer paso fundamental en cualquier proyecto de ciencia de datos.